# Getting data into and out of Neuroglancer

As part of data analysis, we often need to use neuroglancer to help annotate or provide context for data analysis.

Here, we describe the datasets available on [Microns Explorer](https://microns-explorer.org/), explain how to get information out of neuroglancer states, and put the results of some analysis back into neuroglancer.

In [ ]:
import pandas as pd
import numpy as np


from nglui import statebuilder

## 1. Data on Microns Explorer

Several useful data tables are available Under the Data -> Layer 2/3 page of Microns Explorer, which you can [go directly to here](https://microns-explorer.org/phase1).
Due to the nature of connectomics data, the so-called "Phase 1" dataset underwent considerable proofreading and our data tables were updated along the way. The data release "v185" refers to the final state of proofreading before moving on to other tasks.
At the v185 stage, all neurons with cell bodies in the volume had been extensively proofread, as well as Chandelier cell axons.

In our work, we generally use [Pandas](https://pandas.pydata.org/) dataframes for organizing our data.
If you aren't familiar with Pandas, there is [extensive documentation here](https://pandas.pydata.org/docs/user_guide/index.html).
In short, dataframes are a data representation where each row is a data point and each column is one property of the data.

### Soma/Cell Type Table

It is important to know what cells you are working with.
The soma table has basic information about the location and basic cell type (Excitory, Inhibitory, or Glia) of all cells with soma in the volume.
This can be used to look up specific neurons by cell type or provide relative location of positions in the dataset to a cell body.

In [ ]:
soma_table_file = 'data/pni_synapses_v185.csv'
soma_df = pd.read_csv(soma_table_file)

In [ ]:
soma_df.head()

Each row describes a different cell.
Let's look at each column:

**Row and cell type**
* `id`: Unique annotation id for the synapse
* `cell_type`: Valence of each cell: `e` for excitatory, `i` for inhibitory, `g` for glia.
* `pt_root_id`: Unique id for the cell.

**Location in nanometer coordinates**
* `soma_x_nm`, `soma_y_nm`, `soma_z_nm`: The x/y/z location of the center of the cell body in nanometer coordinates. Note that neuroglancer uses voxels, not nanometers.

**Location in voxel coordinates**
* `pt_position`: A 3-element location of the presynaptic site in voxel coordinates. Note that loaded this way, this is imported as a string.

---

For convenience, let's turn the `pt_position` string into a numeric list with some simple python

In [ ]:
# Convert pt_position strings to a list:

def position_string_to_array(pos_string):
    vals = pos_string[1:-1].split(' ')
    return [int(x) for x in vals if len(x) > 0]

# "apply" will apply this function to every element of the column
soma_df['pt_position'] = soma_df['pt_position'].apply(position_string_to_array)

---
#### How do these values relate to what you see in Neuroglancer?

Let's look at a random row from the data perspective and in neuroglancer.
To open a new Neuroglancer window, go to http://layer23.microns-explorer.org.

First, copy the coordinate in pt_position and paste it into the location field in the upper left of the neuroglancer window.

In [ ]:
soma_df.loc[[10]]

This will put you right in the middle of the nucleus.
If you double click it, you'll find that, as expected from the `cell_type` being `e`, this is a pyramidal cell as evident from the dendritic spines.
Moreover, if you look at the object id on the "rendering" tab on the right, this will match the value in `pt_root_id`.

Consistent root ids across data tables and neuroglancer let us keep track of cells.

### Synapse Table

The biggest data table is the synapse table (370 MB), which has a row for each of the 3.2 million synapses in the data set.
Here, we explain what is in this data file.


Here, each row is a synapse detected by the machine learning method described in [Turner et al. 2019](https://arxiv.org/abs/1904.09947).
The data in this table has approximately 90% precision and 90% recall, but has not been explicitly proofread.
As with any output of a machine learning pipeline, this means there are some false positives and false negatives in the data.
If you run across something very surprising, it's always valuable to double check the raw data.

In [ ]:
# Load the synapse table into pandas
synapse_table_file = 'data/pni_synapses_v185.csv'
synapse_df = pd.read_csv(synapse_table_file)
synapse_df.head()

Each row describes the a different aspect of the synapse.
Let's look at each column:

**Annotation and connectivity**
* `id`: Unique annotation id for the synapse
* `pre_root_id`: Object id of the presynaptic neuron
* `post_root_id`: Object id of the postsynaptic neuron

**Synapse size**
* `cleft_vx`: Size of the synaptic cleft in total voxel count. Generally proportional to surface area.

**Location in nanometer coordinates**
* `ctr_pt_x_nm`, `ctr_pt_y_nm`, `ctr_pt_z_nm`: The x/y/z location of the synaptic cleft in nanometer coordinates. Note that neuroglancer uses voxels, not nanometers.

**Location in voxel coordinates**
* `pre_pt_x_vx`, `pre_pt_y_vx`, `pre_pt_z_vx`: The x/y/z location of the presynaptic site in voxel coordinates.
* `ctr_pt_x_vx`, `ctr_pt_y_vx`, `ctr_pt_z_vx`: The x/y/z location of the synaptic cleft in voxel coordinates.
* `post_pt_x_vx`, `post_pt_y_vx`, `post_pt_z_vx`: The x/y/z location of the postsynaptic site in voxel coordinates.

---
#### How do these values relate to what you see in Neuroglancer?

Go to http://layer23.microns-explorer.org.

Let's bring up one synaptic input onto the cell we looked up before.
We do this by selecting all synapses whose `post_root_id` is the root id in question.

In [ ]:
# Cell id copied from above
root_id = 648518346349539095

# Pandas has a handy `query` function that makes selecting data particularly easy
synapse_df.query('post_root_id == @root_id' )

Let's go to the location for one of the synapses.
For simplicity's sake, let's focus on one random row.
First, let's look at the center points.
Copy the values from `ctr_pos_x_vx` to `ctr_pos_z_vx` for one of the rows and paste it into neuroglancer.
If you click the segmentation on the other side of the synapse, it has an id that matches the `pre_root_id` value.
Try again with the `pre_pos_x/y/z_vx` and `post_pos_x/y/z_vx` and you'll see that they go to points within the segmentation on either side.

In [ ]:
synapse_df.query('post_root_id == @root_id' ).iloc[[1320]]

### Proofread Soma Subgraph Synapses

This is a subset of the synapse table that is only synapses between excitatory neurons with cells in the volume.
This has only 1962 synapses, but all of them have been manually inspected and have a spine head volume associated with them.
From a data standpoint, it is similar to the synapse table and thus we will not go into detail here in the tutorial, but we encourage you to look at it if those properties sound useful.

### Making your own annotations in neuroglancer

You can use "export csv" button in the annotation tab to save manual annotations to your computer.
By default, annotations just have points associated with them.
If you want to also get the root id of the objects that you click on, after you create an annotation layer go to the top of its panel and set "Linked Segmentation" to the name of the segmentation layer.

Note that not every annotation uses every column.
Some annotations like lines or bounding boxes have multiple points associated with them, and an ellipsoid annotation has a three radius dimensions.

In [ ]:
annotation_filename = 'data/annotations_spines.csv'
manual_df = pd.read_csv(annotation_filename)

manual_df.head()

---
## Manipulating data and viewing it in Neuroglancer

Visualizing data in python using packages like matplotlib, seaborn, and plotly is fantastic and well-documented.
However, one useful thing is to visualize spatial data in neuroglancer itself, which lets us quickly look at the context of data points in terms of imagery, morphology, and all of the rest of the data we can explore there.

Let's start by digging into inhibitory neurons a bit.

In [ ]:
# First, let's use the cell type table to get the collection of inhibitory cells.
soma_df.query('cell_type == "i"')

### Using StateBuilder to make custom Neuroglancer links

While we could jump to pulling those up in neuroglancer one by one, let's do something a little fancier.
We will make a neuroglancer link that will have the cell bodies of all inhibitory neurons.

StateBuilder in the [nglui package](https://github.com/seung-lab/NeuroglancerAnnotationUI) is designed to turn dataframes into annotations and selected neurons in Neuroglancer.

You were introduced to Neuroglancer through each layer.
StateBuilder works the same way, configuring each layer and mapping rules from dataframe columns to Neuroglancer properties.
We start by configuring the image layer and the segmentation layer.
For the most basic situations like this, all we have to know is the cloud-path to the data sources that neuroglancer uses.

In [ ]:
from nglui import statebuilder

image_path = 'precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked'
segmentation_path = 'precomputed://gs://microns_public_datasets/pinky100_v185/seg'

img = statebuilder.ImageLayerConfig(image_path)
seg = statebuilder.SegmentationLayerConfig(segmentation_path)

We can now build a simple Statebuilder object out of a list of these layer configurations.

In [ ]:
sb = statebuilder.StateBuilder([img, seg])

StateBuilder separates the configuration rules from the rendering.
In this case, we haven't added any data-driven options to the configuration, so we can render out the state immediately.
The `render_as` argument lets us choose to return an HTML link, a long URL, or a dict describing the state.
All are useful in different situations, but for manual exploration `html` is cleanest.

The link below should bring you into the center of the data with nothing selected. Try it!

In [ ]:
sb.render_state(return_as='html')

Now let's add the soma location and root id of each inhibitory neuron.
We have already defined the image and segmentation layers, so we only need to add an annotation layer.
However, we can't just add an annotation layer, we also need to specify how to map data to annotations.
In StateBuilder, this is handled through different Mapper classes: `PointMapper`, `LineMapper`, `SphereMapper`, and `BoundingBoxMapper`.
We will use a `PointMapper` so that each soma location gets one point.

We are going to eventually pass a dataframe to the statebuilder.
A `PointMapper` object needs to know which column of the dataframe contains points.
Note that Neuroglancer expects locations in voxel coordinates, not nanometers.

It would also be convenient to put the root id of each cell in as well.
For the PointMapper, this involves setting the `linked_segmentation_column`.
However, the annotation layer also has to know where to look up root ids from, so you have to specify the name of the segmentation layer as well in the `AnnotationLayerConfig`.

Now we need to add an annotation layer using an `AnnotationLayerConfig` and set the mapping rules to be the PointMapper.
We can then make a new statebuilder with this third layer as well.

In [ ]:
soma_df.head()

In [ ]:
points = statebuilder.PointMapper('pt_position', linked_segmentation_column='pt_root_id')
anno_layer = statebuilder.AnnotationLayerConfig('soma_points', mapping_rules=points, linked_segmentation_layer=seg.name)
sb = statebuilder.StateBuilder([img, seg, anno_layer])

Finally, we pass the dataframe to the StateBuilder on render to show the location of all inhibitory cells.

One handy feature of neuroglancer is using the square brackets `[`/`]` to move between annotations in a list.
If a linked segmentation is set, as it is here, Neuroglancer will also download the mesh so you can quickly visualize the data.

In [ ]:
sb.render_state(soma_df.query('cell_type=="i"'), return_as='html')

The number and properties of the annotations are drawn from the dataframe.
The exact same configuration can be used to show excitatory cells just by changing the dataframe.

In [ ]:
sb.render_state(soma_df.query('cell_type=="e"'), return_as='html')

### Pulling it together to do some analysis

* Use groupby to get a list of output synapses per inhibitory cell.
* Use merge to bring soma position into the same dataframe as cells and compute Euclidean distance from soma.
* Use groupby to 

In [ ]:
inhib_syn_df = synapse_df.query('pre_root_id in @inhibitory_root_ids')

In [ ]:
syn_count_df = inhib_syn_df[['pre_root_id', 'cleft_vx']].groupby('pre_root_id').count()
syn_count_df = syn_count_df.rename(columns={'cleft_vx': 'num_pre'})

In [ ]:
syn_count_df.query('num_pre > 100')

In [ ]:
high_output_inhibitory_root_ids = syn_count_df.query('num_pre > 100').index

In [ ]:
inhib_syn_soma_df = inhib_syn_df.merge(soma_df[['pt_root_id', 'pt_position']], left_on='post_root_id', right_on='pt_root_id', how='inner')

In [ ]:
def synapse_dist_from_soma(row):
    syn_loc = np.array([row['ctr_pt_x_nm'], row['ctr_pt_y_nm'], row['ctr_pt_z_nm']])
    soma_loc = np.array(row['pt_position']) * [4,4,40]
    return np.linalg.norm(syn_loc-soma_loc)

In [ ]:
inhib_syn_soma_df['dist_from_soma'] = inhib_syn_soma_df.apply(synapse_dist_from_soma, axis=1)

In [ ]:
inhib_syn_soma_df['is_soma_synapse'] = inhib_syn_soma_df['dist_from_soma'] < 10_000

In [ ]:
inhib_syn_soma_count = inhib_syn_soma_df.groupby(['pre_root_id', 'is_soma_synapse']).count().loc[high_output_inhibitory_root_ids]

In [ ]:
inhib_syn_soma_count

In [ ]:
root_id = 648518346349539215
inhib_syn_soma_df.query('pre_root_id == @root_id and is_soma_synapse==True')

In [ ]:
inhib_syn_soma_df['ctr_position'] = inhib_syn_soma_df.apply(lambda x: [x['ctr_pos_x_vx'], x['ctr_pos_y_vx'], x['ctr_pos_z_vx']], axis=1)

In [ ]:
img = statebuilder.ImageLayerConfig(image_path)
seg = statebuilder.SegmentationLayerConfig(segmentation_path, selected_ids_column='pre_root_id')

points = statebuilder.PointMapper('ctr_position', linked_segmentation_column='post_root_id')
anno_layer = statebuilder.AnnotationLayerConfig('soma_synapses', mapping_rules=points, linked_segmentation_layer=seg.name)
sb = statebuilder.StateBuilder([img, seg, anno_layer])

In [ ]:
root_id = 648518346349539215
sb.render_state(inhib_syn_soma_df.query('pre_root_id == @root_id and is_soma_synapse == True'), return_as='html')

In [ ]:
root_id = 648518346349539215
sb.render_state(inhib_syn_soma_df.query('pre_root_id == @root_id and is_soma_synapse == False'), return_as='html')

### Using Dash to integrate exploratory analysis and statebuilder into interactive plots

In [ ]:
soma_root_ids = soma_df['pt_root_id'].values

In [ ]:
soma_synapse_df = synapse_df.query('pre_root_id in @soma_root_ids or post_root_id in @soma_root_ids')

In [ ]:
import dash
from dashdataframe import configure_app
# initialize a dash app
app = dash.Dash()

# configure the app
configure_app(app, df)

# run the dash server
app.run_server(port=8880)
